In [1]:
!pip install -q flwr[simulation]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 30.1 MB/s eta 0:00:00


In [1]:
import flwr as fl
from flwr.common import Metrics
from flwr.common.typing import NDArrays, Scalar
from collections import OrderedDict
from typing import List, Tuple, Dict, Optional

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

import os
import librosa

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')#, force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd /content/gdrive/MyDrive/daic/ProjectPrototype

/content/gdrive/MyDrive/daic/ProjectPrototype


In [4]:
!ls

'212148conf copy 3.ipynb'   edaicwoz   MFCCs_1030   MFCCs_1030.zip   preprocess_data.ipynb


In [5]:
train_labels_df = pd.read_csv("edaicwoz/train_split.csv")
test_labels_df = pd.read_csv("edaicwoz/test_split.csv")
val_labels_df = pd.read_csv("edaicwoz/dev_split.csv")

In [6]:
def load_audio_files(data_dir, sr=16000):
    file_ids = os.listdir(data_dir)
    subject_ids = []
    file_paths = []
    types = []
    labels = []
    labels_binary = []

    for file_id in file_ids:
        file_id = file_id.split("_")[0]
        file_path = [data_dir + "/" + file_id + "/" + file_id + "_MFCC_" + str(i) + ".npy" for i in range(len(next(iter(enumerate(os.walk(data_dir + "/" + str(file_id) + "/"))))[1][2]))]
        if int(file_id) in train_labels_df["Participant_ID"].values:
            types.append(0)
            labels.append(train_labels_df[train_labels_df["Participant_ID"] == int(file_id)]['PHQ_Score'].values[0])
            labels_binary.append(train_labels_df[train_labels_df["Participant_ID"] == int(file_id)]['PHQ_Binary'].values[0])

        elif int(file_id) in test_labels_df["Participant_ID"].values:
            types.append(1)
            labels.append(test_labels_df[test_labels_df["Participant_ID"] == int(file_id)]['PHQ_Score'].values[0])
            labels_binary.append(test_labels_df[test_labels_df["Participant_ID"] == int(file_id)]['PHQ_Binary'].values[0])
        else:
            types.append(2)
            labels.append(val_labels_df[val_labels_df["Participant_ID"] == int(file_id)]['PHQ_Score'].values[0])
            labels_binary.append(val_labels_df[val_labels_df["Participant_ID"] == int(file_id)]['PHQ_Binary'].values[0])
        subject_ids.append(int(file_id))
        file_paths.append(file_path)

    return file_ids, subject_ids, file_paths, types, labels, labels_binary

data_dir = "MFCCs_1030"

file_ids, subject_ids, file_paths, types, labels, labels_binary = load_audio_files(data_dir)

In [7]:
def prepare_audio_set(file_paths):

    samples = []
    samples_ids = []
    samples_types = []
    samples_labels = []
    samples_labels_binary = []

    for i, file_path in enumerate(file_paths):
        all_mfccs = []
        for j in range(len(file_path)):
            all_mfccs.append(np.load(file_path[j]))
        all_mfccs = np.array(all_mfccs)
        samples.extend(all_mfccs)
        samples_ids.extend([subject_ids[i]] * len(all_mfccs))
        samples_types.extend([types[i]] * len(all_mfccs))
        samples_labels.extend([labels[i]] * len(all_mfccs))
        samples_labels_binary.extend([labels_binary[i]] * len(all_mfccs))

    samples = np.array(samples)

    samples_ids = np.array(samples_ids)
    samples_types = np.array(samples_types)
    samples_labels = np.array(samples_labels)
    samples_labels_binary = np.array(samples_labels_binary)

    return samples, samples_ids, samples_types, samples_labels, samples_labels_binary

print("[INFO] preparing data...")
samples, samples_ids, samples_types, samples_labels, samples_labels_binary = prepare_audio_set(file_paths)
samples = np.swapaxes(samples, 1, 2)

[INFO] preparing data...


In [8]:
training_samples = samples[samples_types == 0]
training_labels = samples_labels_binary[samples_types == 0]
training_subject_ids = samples_ids[samples_types == 0]

test_samples = samples[samples_types == 1]
test_labels = samples_labels_binary[samples_types == 1]

val_samples = samples[samples_types == 2]
val_labels_df = samples_labels_binary[samples_types == 2]

In [9]:
del samples, samples_ids, samples_types, samples_labels, samples_labels_binary
del file_ids, subject_ids, file_paths, types, labels, labels_binary

In [10]:
SEED = 42
NUM_CLIENTS = 4
BATCH_SIZE = 32
NUM_ROUNDS = 5
DEPRESSIVE_MULTIPLIER = 30
NON_DEPRESSIVE_MULTIPLIER = 10

In [11]:
def partition_data(X: np.ndarray, X_ids: np.ndarray, n_clients: int, d_mult: int, n_d_mult: int) -> Tuple[List[np.ndarray], List[np.ndarray]]:

    unique_subject_ids, counts = np.unique(X_ids, return_counts=True)

    # grouping the training samples by patient
    training_samples_grouped = []
    for i in unique_subject_ids:
        training_samples_grouped.append(X[X_ids == i])
    training_samples_grouped = np.array(training_samples_grouped)

    mask_30_segments = np.array([sample.shape[0] == DEPRESSIVE_MULTIPLIER for sample in training_samples_grouped])

    # creating masks to get deppressives and non deppressives
    mask_10_segments = np.array([sample.shape[0] == NON_DEPRESSIVE_MULTIPLIER for sample in training_samples_grouped])

    data_array_30_segments = training_samples_grouped[mask_30_segments]

    data_array_10_segments = training_samples_grouped[mask_10_segments]

    # recreating labels
    X_train_zeros = np.array_split(data_array_10_segments, NUM_CLIENTS)
    X_train_ones = np.array_split(data_array_30_segments, NUM_CLIENTS)

    # concatenating splitted ones and zeros with labels
    X_train_splitted = [] # (NUM_CLIENTS, data)
    y_train_splitted = [] # (NUM_CLIENTS, labels)
    for i in range(NUM_CLIENTS):

        # stack the segments from groups then concatenate
        client_data = np.concatenate((np.vstack(X_train_zeros[i]), np.vstack(X_train_ones[i])), axis=0)
        client_labels = np.concatenate((np.zeros((X_train_zeros[i].shape[0] * NON_DEPRESSIVE_MULTIPLIER), dtype=int), np.ones((X_train_ones[i].shape[0] * DEPRESSIVE_MULTIPLIER), dtype=int)), axis=0)

        X_train_splitted.append(client_data)
        y_train_splitted.append(client_labels)

    return X_train_splitted, y_train_splitted

X_trains, y_trains = partition_data(training_samples, training_subject_ids, NUM_CLIENTS, DEPRESSIVE_MULTIPLIER, NON_DEPRESSIVE_MULTIPLIER)

<ipython-input-11-466285f35f26>:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training_samples_grouped = np.array(training_samples_grouped)


In [12]:
del training_samples, training_labels, training_subject_ids

In [13]:
X_trains[0].shape

(610, 15001, 13)

In [14]:
def get_model(input_shape):
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.GRU(units = 64, input_shape = input_shape))

    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.3))

    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    return model

In [26]:
from flwr.common.typing import NDArrays
# creating clients code
class FlowerClient(fl.client.NumPyClient):

    def __init__(self, model: tf.keras.models.Sequential, X_train: np.ndarray, y_train: np.ndarray):
        self.model = model
        self.X_train = X_train
        self.y_train = y_train

    def get_parameters(self, config):
        return self.model.get_weights()


    def fit(self, parameters: NDArrays, config: Dict[str, Scalar]) -> NDArrays:

        # compile the model
        self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1), metrics=['accuracy'])

        # assign weights
        self.model.set_weights(parameters)

        # train for the client using its data
        history = self.model.fit(self.X_train, self.y_train , epochs=1, verbose=0)
        results = {
            "loss": history.history["loss"][0],
            "accuracy": history.history["accuracy"][0],
        }
        return self.model.get_weights(), len(self.X_train), results

    def evaluate(self, parameters: NDArrays, config: Dict[str, Scalar])-> Tuple[float, int, Dict[str, Scalar]]:
        return 1.2, len(self.X_train), {"accuracy": 0}

In [27]:
# client creator by client id
def create_client_fn(cid: str) -> FlowerClient:

    input_shape = (15001, 13)
    model = get_model(input_shape)
    cid_int = int(cid)
    return FlowerClient(model, X_trains[cid_int], y_trains[cid_int])

In [28]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    return {"accuracy": 0}

In [ ]:
best_accuracy = 0.0
best_loss = 999
weights = np.array([])
def evaluate(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
    ) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    """Centralized evaluation function"""

    input_shape = (15001, 13)
    model = get_model(input_shape)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1), metrics=['accuracy'])
    model.set_weights(parameters)

    loss, accuracy = model.evaluate(val_samples, val_labels_df, batch_size=16, verbose=0)

    global best_accuracy
    global best_loss
    global weights

    if loss < best_loss:
        best_accuracy = accuracy
        weights = parameters
        best_loss = loss

    return loss, {"accuracy": accuracy}


client_resources = {"num_cpus": 2}
if tf.config.get_visible_devices("GPU"):
    client_resources["num_gpus"] = 1

# Specify the Strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # Sample 100% of available clients for training
    fraction_evaluate=0.0,
    min_fit_clients=NUM_CLIENTS,
    min_available_clients=NUM_CLIENTS,  # Wait until all 8 clients are available
    evaluate_metrics_aggregation_fn=weighted_average,
    evaluate_fn=evaluate
)

# Start simulation
history = fl.simulation.start_simulation(
    client_fn=create_client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=25),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-12-10 17:02:47,803 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=25, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=25, round_timeout=None)
2023-12-10 17:02:52,004	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-12-10 17:02:53,551 | app.py:213 | Flower VCE: Ray initialized with resources: {'memory': 32677203150.0, 'object_store_memory': 16338601574.0, 'accelerator_type:V100': 1.0, 'node:__internal_head__': 1.0, 'CPU': 8.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'memory': 32677203150.0, 'object_store_memory': 16338601574.0, 'accelerator_type:V100': 1.0, 'node:__internal_head__': 1.0, 'CPU': 8.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0}
INFO flwr 2023-12-10 17:02:53,553 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO:flwr:Optimize your simulation with Flower V

In [30]:
history

History (loss, centralized):
	round 0: 0.7060863375663757
	round 1: 0.7022933959960938
	round 2: 0.7004526257514954
	round 3: 0.6980700492858887
	round 4: 0.6964865326881409
	round 5: 0.6952226758003235
History (metrics, centralized):
{'accuracy': [(0, 0.4962500035762787), (1, 0.4950000047683716), (2, 0.48625001311302185), (3, 0.5062500238418579), (4, 0.5149999856948853), (5, 0.5149999856948853)]}

In [ ]:
# printing the validation results
print(best_accuracy)
print(weights)